# NB-2 Data Access and Wrangling

In [1]:
# deal with some cruft on my machine
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

load tools

In [2]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pk
import censusdata
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

A Census Block Group is a geographical unit used by the United States Census Bureau which is between the Census Tract and the Census Block. It is the smallest geographical unit for which the bureau publishes sample data, i.e. data which is only collected from a fraction of all households.

In [3]:
buncombe_bgs = censusdata.geographies(censusdata.censusgeo([('state', '37'), ('county', '021'),('block group', '*')]), 'acs5', '2015')

In [4]:
print("number of block groups {}".format(len(buncombe_bgs)))

number of block groups 154



Now that we have identified the variables and geographies of interest, we can download the data using censusdata.download and compute variables for the percent unemployed and the percent with no high school degree. This code comes from the example. Note we are pulling 2015 data.

In [5]:
buncombebg = censusdata.download('acs5', '2015',
                             censusdata.censusgeo([('state', '37'), ('county', '021'), ('block group', '*')]),
                             ['B23025_003E', 'B23025_005E', 'B15003_001E', 'B15003_002E', 'B15003_003E',
                              'B15003_004E', 'B15003_005E', 'B15003_006E', 'B15003_007E', 'B15003_008E',
                              'B15003_009E', 'B15003_010E', 'B15003_011E', 'B15003_012E', 'B15003_013E',
                              'B15003_014E', 'B15003_015E', 'B15003_016E'])
buncombebg['percent_unemployed'] = buncombebg.B23025_005E / buncombebg.B23025_003E * 100
buncombebg['percent_nohs'] = (buncombebg.B15003_002E + buncombebg.B15003_003E + buncombebg.B15003_004E
                          + buncombebg.B15003_005E + buncombebg.B15003_006E + buncombebg.B15003_007E + buncombebg.B15003_008E
                          + buncombebg.B15003_009E + buncombebg.B15003_010E + buncombebg.B15003_011E + buncombebg.B15003_012E
                          + buncombebg.B15003_013E + buncombebg.B15003_014E +
                          buncombebg.B15003_015E + buncombebg.B15003_016E) / buncombebg.B15003_001E * 100
buncombebg = buncombebg[['percent_unemployed', 'percent_nohs']]
buncombebg.describe()

,percent_unemployed,percent_nohs
count,154.00,154.00
mean,7.24,10.81
std,5.52,8.16
min,0.00,0.00
25%,3.41,4.62
50%,6.70,8.70
75%,9.84,15.14
max,30.14,42.00


save data to pickle and csv file for exploration and modeling

In [6]:
buncombebg.to_pickle('uemp_nohs.pk')
buncombebg.to_csv('uemp_nohs.csv')